In [1]:
import math as m
import numpy as np
import random as r
import matplotlib.pyplot as plt
import sys

In [2]:
import torch
from torch import nn
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR

In [3]:
from nflows.flows.base import Flow
from nflows.distributions.uniform import BoxUniform
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedPiecewiseRationalQuadraticAutoregressiveTransform
from nflows.transforms.autoregressive import MaskedPiecewiseQuadraticAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation
from nflows.transforms.permutations import RandomPermutation
from nflows.transforms.splines.rational_quadratic import rational_quadratic_spline
from torch.utils.tensorboard import SummaryWriter

In [4]:
import subprocess
import time
import os
from copy import deepcopy
import math as m
import gc

## Tensorboard writer for loss logging

In [5]:
writer = SummaryWriter()

## GPU/CPU selection

In [6]:
device = torch.device("cuda:1")
#device = torch.device("cpu")

## Reweighting strategy - must be min, mean or max

In [7]:
reference_method = 'max'

## Hyperparameters

In [8]:
n_RQS_knots = 10   # Number of knots in RQS transform
n_made_layers = 1  # Number of hidden layers in every made network
n_made_units = 100 # Number of units in every layer of the made network
n_flow_layers = 6  # Number of layers in the flow

batch_size = 1024
n_epochs = 800
adam_lr = 0.001     # Learning rate for the ADAM optimizer (default: 0.001)

n_train = int(1e6)  # Number of training events
n_test = int(1e5)   # Number of testing events
n_sample = int(1e6) # Number of samples for ess evaluation

## Load the train data and reweight

In [9]:
# Load
train_samples = np.genfromtxt("data/weighted_samples.csv", delimiter=',')[:n_train]
train_weights = np.genfromtxt("data/weighted_weights.csv", delimiter=',')[:n_train]

# Get reference weight
reference_methods = {'min': np.amin, 'mean': np.mean, 'max': np.amax}
ref_weight = reference_methods[reference_method](train_weights)

# Reweighting and rejection sampling
train_weights = train_weights / ref_weight
p_rejection_sampling = np.random.rand(len(train_weights))
select = p_rejection_sampling < train_weights

train_samples = train_samples[select]
train_weights = train_weights[select]
train_weights[train_weights < 1.0] = 1.0

# Normalize weights
train_weights /= train_weights.mean()

# Convert to torch tensors
train_samples = torch.tensor(train_samples, dtype=torch.float32, device=device)
train_weights = torch.tensor(train_weights, dtype=torch.float32, device=device)

## Load the test data

In [10]:
test_samples = torch.tensor(np.genfromtxt("data/unweighted_samples.csv", delimiter=',')[:n_test], dtype=torch.float32, device=device)

## Set up the flow

In [11]:
event_dim = train_samples.shape[1]
base_dist = BoxUniform(torch.zeros(event_dim), torch.ones(event_dim))

transforms = []
for _ in range(n_flow_layers):
    transforms.append(RandomPermutation(features=event_dim))
    transforms.append(MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        features=event_dim, 
        hidden_features=n_made_units,
        num_bins=n_RQS_knots,
        num_blocks=n_made_layers-1,
        tails="constrained",
        use_residual_blocks=False
    ))
transform = CompositeTransform(transforms)

flow = Flow(transform, base_dist).to(device)
optimizer = optim.Adam(flow.parameters(), lr=adam_lr)

scheduler = MultiStepLR(optimizer, milestones=[350, 425, 500, 575, 650, 725, 800], gamma=0.5)

## Training

In [12]:
data_size = train_samples.shape[0]
n_batches = m.ceil(data_size/batch_size)

data_size_validation = test_samples.shape[0]
n_batches_validate = m.ceil(data_size_validation/batch_size)

best_validation_loss = np.inf
best_ess = 0

for epoch in range(n_epochs):
    
    permutation = torch.randperm(data_size, device=device)    

    # Loop over batches
    cum_loss = 0
    for batch in range(n_batches):
        # Set up the batch
        batch_begin = batch*batch_size
        batch_end   = min( (batch+1)*batch_size, data_size-1 )
        indices = permutation[batch_begin:batch_end]
        samples_batch = train_samples[indices]
        weights_batch = train_weights[indices]
        
        # Take a step
        optimizer.zero_grad()
        loss = -(flow.log_prob(inputs=samples_batch)*weights_batch).mean()
        loss.backward()
        optimizer.step()

        # Compute cumulative loss
        cum_loss = (cum_loss*batch + loss.item())/(batch+1)

        if batch%25 == 0:
            print("epoch = ", epoch, "batch = ", batch, "/", n_batches, "loss = ", cum_loss)
    
    writer.add_scalar("Loss_train", cum_loss, epoch)
    scheduler.step()
    

    # ---------- Compute validation loss -----------
    validation_loss = 0
    for batch in range(n_batches_validate):
        batch_begin = batch*batch_size
        batch_end = min( (batch+1)*batch_size, data_size_validation-1 )
        samples_batch = test_samples[batch_begin:batch_end]
    
        with torch.no_grad():
            validation_loss = (validation_loss*batch - (flow.log_prob(samples_batch)).mean())/(batch+1)

    print("Validation loss = ", validation_loss.item())
    writer.add_scalar("Loss_test", validation_loss.item(), epoch)

    if validation_loss < best_validation_loss:
        torch.save(flow, "flow_model_weighted_{}_best_validation.pt".format(reference_method))
        best_validation_loss = validation_loss

    
    # ---------- Compute effective sample size ----------
    # generate samples and evaluate llhs
    with torch.no_grad():
        samples = flow.sample(n_sample)
        llhs = flow.log_prob(samples)

    # Store files
    np.savetxt("/tmp/samples_weighted_file.csv", samples.cpu().numpy(), delimiter=',')
    np.savetxt("/tmp/llhs_weighted_file.csv", np.exp(llhs.cpu().numpy()), delimiter=',')

    # Run the evaluator
    cmd = os.path.abspath(os.getcwd())+'/ME_VEGAS/compute_metrics_from_likelihoods /tmp/samples_weighted_file.csv /tmp/llhs_weighted_file.csv'
    b = subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE).stdout
    lines = b.decode('ascii').split("\n")

    ess = float(lines[2].split(' ')[-1])
    
    print("Effective sample size = ", ess)
    writer.add_scalar("Effective_sample_size", ess, epoch)

    if ess > best_ess:
        torch.save(flow, "flow_model_weighted_{}_best_ess.pt".format(reference_method))
        best_ess = ess
        
torch.save(flow, "flow_model_weighted_{}_final.pt".format(reference_method))

epoch =  0 batch =  0 / 63 loss =  0.5265049934387207
epoch =  0 batch =  25 / 63 loss =  -4.674710771212211
epoch =  0 batch =  50 / 63 loss =  -7.249040857249615
Validation loss =  -15.671630859375
Effective sample size =  0.000161148
epoch =  1 batch =  0 / 63 loss =  -15.70964241027832
epoch =  1 batch =  25 / 63 loss =  -18.273696239177994
epoch =  1 batch =  50 / 63 loss =  -19.774563546274212
Validation loss =  -22.197216033935547
Effective sample size =  0.038571
epoch =  2 batch =  0 / 63 loss =  -22.309574127197266
epoch =  2 batch =  25 / 63 loss =  -22.544276604285606
epoch =  2 batch =  50 / 63 loss =  -22.68653211406633
Validation loss =  -22.896236419677734
Effective sample size =  0.127785
epoch =  3 batch =  0 / 63 loss =  -23.150585174560547
epoch =  3 batch =  25 / 63 loss =  -23.017963849581204
epoch =  3 batch =  50 / 63 loss =  -22.95378928091012
Validation loss =  -23.099830627441406
Effective sample size =  0.199399
epoch =  4 batch =  0 / 63 loss =  -23.2873573

epoch =  34 batch =  25 / 63 loss =  -23.854330869821403
epoch =  34 batch =  50 / 63 loss =  -23.827235764148192
Validation loss =  -23.669841766357422
Effective sample size =  0.491067
epoch =  35 batch =  0 / 63 loss =  -23.716535568237305
epoch =  35 batch =  25 / 63 loss =  -23.819158920874962
epoch =  35 batch =  50 / 63 loss =  -23.807133356730144
Validation loss =  -23.711997985839844
Effective sample size =  0.528474
epoch =  36 batch =  0 / 63 loss =  -23.878673553466797
epoch =  36 batch =  25 / 63 loss =  -23.8279794546274
epoch =  36 batch =  50 / 63 loss =  -23.786989810420017
Validation loss =  -23.606908798217773
Effective sample size =  0.414813
epoch =  37 batch =  0 / 63 loss =  -23.594127655029297
epoch =  37 batch =  25 / 63 loss =  -23.8110199708205
epoch =  37 batch =  50 / 63 loss =  -23.829611983953736
Validation loss =  -23.68359375
Effective sample size =  0.494058
epoch =  38 batch =  0 / 63 loss =  -23.930530548095703
epoch =  38 batch =  25 / 63 loss =  -2

epoch =  68 batch =  25 / 63 loss =  -23.893814233633186
epoch =  68 batch =  50 / 63 loss =  -23.890488680671243
Validation loss =  -23.7078857421875
Effective sample size =  0.511446
epoch =  69 batch =  0 / 63 loss =  -23.851409912109375
epoch =  69 batch =  25 / 63 loss =  -23.899106612572304
epoch =  69 batch =  50 / 63 loss =  -23.889161203421793
Validation loss =  -23.632160186767578
Effective sample size =  0.43597
epoch =  70 batch =  0 / 63 loss =  -23.83512306213379
epoch =  70 batch =  25 / 63 loss =  -23.852391169621395
epoch =  70 batch =  50 / 63 loss =  -23.8961139753753
Validation loss =  -23.621028900146484
Effective sample size =  0.437817
epoch =  71 batch =  0 / 63 loss =  -23.885143280029297
epoch =  71 batch =  25 / 63 loss =  -23.791781205397385
epoch =  71 batch =  50 / 63 loss =  -23.8532325146245
Validation loss =  -23.669994354248047
Effective sample size =  0.468255
epoch =  72 batch =  0 / 63 loss =  -23.839658737182617
epoch =  72 batch =  25 / 63 loss = 

epoch =  102 batch =  25 / 63 loss =  -23.909619698157677
epoch =  102 batch =  50 / 63 loss =  -23.92347062802782
Validation loss =  -23.695384979248047
Effective sample size =  0.484409
epoch =  103 batch =  0 / 63 loss =  -23.71314239501953
epoch =  103 batch =  25 / 63 loss =  -23.94952818063589
epoch =  103 batch =  50 / 63 loss =  -23.973639544318704
Validation loss =  -23.69082260131836
Effective sample size =  0.468394
epoch =  104 batch =  0 / 63 loss =  -23.832443237304688
epoch =  104 batch =  25 / 63 loss =  -23.891023855942947
epoch =  104 batch =  50 / 63 loss =  -23.907737170948703
Validation loss =  -23.704669952392578
Effective sample size =  0.483617
epoch =  105 batch =  0 / 63 loss =  -23.930498123168945
epoch =  105 batch =  25 / 63 loss =  -23.933949910677395
epoch =  105 batch =  50 / 63 loss =  -23.952912461523916
Validation loss =  -23.647663116455078
Effective sample size =  0.439676
epoch =  106 batch =  0 / 63 loss =  -23.827497482299805
epoch =  106 batch =

Effective sample size =  0.453826
epoch =  136 batch =  0 / 63 loss =  -24.155799865722656
epoch =  136 batch =  25 / 63 loss =  -24.010983833899864
epoch =  136 batch =  50 / 63 loss =  -23.983442418715534
Validation loss =  -23.56993293762207
Effective sample size =  0.394048
epoch =  137 batch =  0 / 63 loss =  -23.92264175415039
epoch =  137 batch =  25 / 63 loss =  -23.9925898038424
epoch =  137 batch =  50 / 63 loss =  -23.9881399566052
Validation loss =  -23.57893943786621
Effective sample size =  0.36668
epoch =  138 batch =  0 / 63 loss =  -23.878185272216797
epoch =  138 batch =  25 / 63 loss =  -23.949245893038235
epoch =  138 batch =  50 / 63 loss =  -23.978073868097045
Validation loss =  -23.671926498413086
Effective sample size =  0.461837
epoch =  139 batch =  0 / 63 loss =  -24.088685989379883
epoch =  139 batch =  25 / 63 loss =  -23.98609447479248
epoch =  139 batch =  50 / 63 loss =  -23.974415349025353
Validation loss =  -23.703815460205078
Effective sample size =  

...

Effective sample size =  0.338936
epoch =  641 batch =  0 / 63 loss =  -24.385730743408203
epoch =  641 batch =  25 / 63 loss =  -24.299401943500225
epoch =  641 batch =  50 / 63 loss =  -24.30034345739028
Validation loss =  -23.626163482666016
Effective sample size =  0.346454
epoch =  642 batch =  0 / 63 loss =  -24.41001319885254
epoch =  642 batch =  25 / 63 loss =  -24.323838380666878
epoch =  642 batch =  50 / 63 loss =  -24.31231846528894
Validation loss =  -23.62674903869629
Effective sample size =  0.378584
epoch =  643 batch =  0 / 63 loss =  -24.277095794677734
epoch =  643 batch =  25 / 63 loss =  -24.302058366628792
epoch =  643 batch =  50 / 63 loss =  -24.309597800759708
Validation loss =  -23.62357521057129
Effective sample size =  0.362046
epoch =  644 batch =  0 / 63 loss =  -24.191925048828125
epoch =  644 batch =  25 / 63 loss =  -24.27369389167199
epoch =  644 batch =  50 / 63 loss =  -24.29667080149931
Validation loss =  -23.630239486694336
Effective sample size =

epoch =  674 batch =  50 / 63 loss =  -24.305427962658452
Validation loss =  -23.631126403808594
Effective sample size =  0.044512
epoch =  675 batch =  0 / 63 loss =  -24.251554489135742
epoch =  675 batch =  25 / 63 loss =  -24.327580525324894
epoch =  675 batch =  50 / 63 loss =  -24.318885391833735
Validation loss =  -23.629728317260742
Effective sample size =  0.341992
epoch =  676 batch =  0 / 63 loss =  -24.262821197509766
epoch =  676 batch =  25 / 63 loss =  -24.291501778822678
epoch =  676 batch =  50 / 63 loss =  -24.315180423212986
Validation loss =  -23.629947662353516
Effective sample size =  0.3714
epoch =  677 batch =  0 / 63 loss =  -24.15227508544922
epoch =  677 batch =  25 / 63 loss =  -24.27300159747784
epoch =  677 batch =  50 / 63 loss =  -24.313009411680927
Validation loss =  -23.62763023376465
Effective sample size =  0.361385
epoch =  678 batch =  0 / 63 loss =  -24.133901596069336
epoch =  678 batch =  25 / 63 loss =  -24.308438081007736
epoch =  678 batch = 

epoch =  708 batch =  25 / 63 loss =  -24.28345826955942
epoch =  708 batch =  50 / 63 loss =  -24.311298594755286
Validation loss =  -23.628353118896484
Effective sample size =  0.390235
epoch =  709 batch =  0 / 63 loss =  -24.337318420410156
epoch =  709 batch =  25 / 63 loss =  -24.32306172297551
epoch =  709 batch =  50 / 63 loss =  -24.308885275148867
Validation loss =  -23.628915786743164
Effective sample size =  0.292245
epoch =  710 batch =  0 / 63 loss =  -24.240440368652344
epoch =  710 batch =  25 / 63 loss =  -24.329721230726975
epoch =  710 batch =  50 / 63 loss =  -24.308111564785822
Validation loss =  -23.628189086914062
Effective sample size =  0.384989
epoch =  711 batch =  0 / 63 loss =  -24.41765594482422
epoch =  711 batch =  25 / 63 loss =  -24.291439056396484
epoch =  711 batch =  50 / 63 loss =  -24.31165137945437
Validation loss =  -23.628910064697266
Effective sample size =  0.348716
epoch =  712 batch =  0 / 63 loss =  -24.43328857421875
epoch =  712 batch = 

Effective sample size =  0.333876
epoch =  742 batch =  0 / 63 loss =  -24.25192642211914
epoch =  742 batch =  25 / 63 loss =  -24.30359180157001
epoch =  742 batch =  50 / 63 loss =  -24.320946450326957
Validation loss =  -23.626497268676758
Effective sample size =  0.370091
epoch =  743 batch =  0 / 63 loss =  -24.26023292541504
epoch =  743 batch =  25 / 63 loss =  -24.31954112419715
epoch =  743 batch =  50 / 63 loss =  -24.3338114046583
Validation loss =  -23.628210067749023
Effective sample size =  0.363435
epoch =  744 batch =  0 / 63 loss =  -24.287227630615234
epoch =  744 batch =  25 / 63 loss =  -24.31111607184777
epoch =  744 batch =  50 / 63 loss =  -24.314895816877776
Validation loss =  -23.62639617919922
Effective sample size =  0.380502
epoch =  745 batch =  0 / 63 loss =  -24.29705047607422
epoch =  745 batch =  25 / 63 loss =  -24.317374742948093
epoch =  745 batch =  50 / 63 loss =  -24.30817147797229
Validation loss =  -23.627363204956055
Effective sample size =  0

epoch =  775 batch =  50 / 63 loss =  -24.323148764815986
Validation loss =  -23.62754249572754
Effective sample size =  0.321149
epoch =  776 batch =  0 / 63 loss =  -24.342388153076172
epoch =  776 batch =  25 / 63 loss =  -24.311338424682617
epoch =  776 batch =  50 / 63 loss =  -24.305698319977406
Validation loss =  -23.62751007080078
Effective sample size =  0.327603
epoch =  777 batch =  0 / 63 loss =  -24.44522476196289
epoch =  777 batch =  25 / 63 loss =  -24.330693391653213
epoch =  777 batch =  50 / 63 loss =  -24.315040850171854
Validation loss =  -23.62718963623047
Effective sample size =  0.362752
epoch =  778 batch =  0 / 63 loss =  -24.00171661376953
epoch =  778 batch =  25 / 63 loss =  -24.331462933466984
epoch =  778 batch =  50 / 63 loss =  -24.308538362091664
Validation loss =  -23.627901077270508
Effective sample size =  0.393212
epoch =  779 batch =  0 / 63 loss =  -24.375083923339844
epoch =  779 batch =  25 / 63 loss =  -24.35012986109807
epoch =  779 batch =  